In [22]:
import os
import pandas as pd
from pandas import read_csv, read_excel, DataFrame
from sae import COTSCar
import gurobipy as gp
from gurobipy import GRB
from gurobipy import quicksum as qsum
import numpy as np
from typing import List

import sys
sys.path.append('../')
import saedfsc

Price of the car.

In [23]:
maxProductPrice = 400000

Information about customers

In [24]:
customers = saedfsc.customers['Name'].to_list()
cQty = dict(zip(saedfsc.customers['Name'], saedfsc.customers['Quantity'])) # customer quantities
cPriceFocus = dict(zip(saedfsc.customers['Name'], saedfsc.customers['PriceFocus']))
name_weights_partworth_dict = saedfsc.customers.set_index('Name')['PartworthUtilityWeights'].to_dict()
name_weights_perf_dict = saedfsc.customers.set_index('Name')['PerformanceUtilityWeights'].to_dict()
cWtsPartworth = {c : np.fromstring(name_weights_partworth_dict[c].strip('[]'), sep=',') for c in name_weights_partworth_dict}
cWtsPerf = {c : np.fromstring(name_weights_perf_dict[c].strip('[]'), sep=',') for c in name_weights_perf_dict}

Methods to help compute the market share for each customer type (based on logit model).

In [25]:
def getTotalUtilityForCustomer(car : COTSCar, c : str, utilFn = 'partworth'):
    total_utility = 0
    pricePerf = (maxProductPrice - car.price) / maxProductPrice # Normalized price (0 is best, 1 is worst)
    if utilFn == 'partworth':
        product_utility = car.partworth_objectives(weights=cWtsPartworth[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility - cPriceFocus[c]*pricePerf
    elif utilFn == 'performance':
        product_utility = car.objectives(weights=cWtsPerf[c])[0]
        total_utility += (1-cPriceFocus[c])*product_utility + cPriceFocus[c]*pricePerf
    return total_utility

def getMarketShare(car : COTSCar, c : str, competitors : List[COTSCar], utilFn = 'partworth'): # based on logit model of demand
    carUtility = getTotalUtilityForCustomer(car, c, utilFn)
    totalCompetitorUtility = sum([getTotalUtilityForCustomer(competitorCar, c, utilFn) for competitorCar in competitors])
    return carUtility / (totalCompetitorUtility + carUtility)

Create a set of competitors.

In [26]:
profitMargin = 0.1
competitors = []
for i in range(3):
    competitorCar = COTSCar()
    competitorCar.price = (1+profitMargin)*competitorCar.cost()
    competitorCar.name = "Competitor Car " + str(i)
    competitors.append(competitorCar)

names = [car.name for car in competitors]
designs = [car.vector for car in competitors]
salesPrices = ['$' + format(car.price, ",.2f") for car in competitors]
costs = ['$' + format(car.cost(), ",.2f") for car in competitors]
competitorsDF = pd.DataFrame({
    'Name': names,
    'Design': designs,
    'SalesPrice': salesPrices,
    'ProductCost': costs
})
competitorsDF

,Name,Design,SalesPrice,ProductCost
0,Competitor Car 0,"[11, 5, 1, 1, 4, 4, 1, 11, 10, 2, 38, 23, 2, 2...","$15,195.75","$13,814.32"
1,Competitor Car 1,"[2, 4, 7, 5, 10, 5, 0, 2, 1, 2, 41, 18, 2, 0, ...","$73,620.37","$66,927.61"
2,Competitor Car 2,"[10, 10, 8, 5, 7, 2, 6, 3, 0, 4, 11, 52, 1, 2,...","$279,880.44","$254,436.76"


## Partworth Utility

Create a random car and compute market share

In [27]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c) for c in customers]
market_shares = [getMarketShare(product, c, competitors) for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors)*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 100,841.03
Cost: $ 91,673.66
Total expected profit: $ 334,899,165.40
Market information for our car:


,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,-0.711540,0.209642,98.322002
1,CustomerType2,929,-0.763950,0.264704,245.909898
2,CustomerType3,294,-0.904846,0.239200,70.324662
3,CustomerType4,834,-0.676633,0.230449,192.194106
4,CustomerType5,733,-0.827020,0.216249,158.510834
5,CustomerType6,35,-1.183427,0.282946,9.903102
6,CustomerType7,239,-1.656241,0.317917,75.982152
7,CustomerType8,564,-0.889819,0.221901,125.152054
8,CustomerType9,927,-0.955588,0.222470,206.230037
9,CustomerType10,366,-0.756482,0.207289,75.867759


## Performance Utility

In [28]:
carDesignVec = [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
product = COTSCar()
product.vector = carDesignVec
product.price = (1 + profitMargin)*product.cost()
print("OUR CAR:") 
print("Design: ", product.vector)
print("Sales price: $", format(product.price, ",.2f"))
print("Cost: $", format(product.cost(), ",.2f"))

qtys = [cQty[c] for c in customers]
utilities = [getTotalUtilityForCustomer(product, c, 'performance') for c in customers]
market_shares = [getMarketShare(product, c, competitors, 'performance') for c in customers]
expected_customer_demand = [getMarketShare(product, c, competitors, 'performance')*cQty[c] for c in customers]

df = pd.DataFrame({
    'Name': customers,
    'Quantity': qtys,
    'Utility for our car': utilities,
    'Market share': market_shares,
    'Expected demand': expected_customer_demand
})
total_profit = product.price*df['Expected demand'].sum()
print("Total expected profit: $", format(total_profit, ",.2f"))
print("Market information for our car:")
df

OUR CAR:
Design:  [12, 7, 6, 12, 9, 1, 0, 13, 9, 1, 39, 26, 2, 1, 33, 11, 4]
Sales price: $ 27,051.26
Cost: $ 24,592.06
Total expected profit: $ 82,782,719.25
Market information for our car:


,Name,Quantity,Utility for our car,Market share,Expected demand
0,CustomerType1,469,1.717565,0.222605,104.401734
1,CustomerType2,929,1.006846,0.295391,274.418532
2,CustomerType3,294,1.361066,0.237104,69.708613
3,CustomerType4,834,1.404835,0.232907,194.244375
4,CustomerType5,733,1.250322,0.242198,177.530827
5,CustomerType6,35,1.855804,0.221871,7.765502
6,CustomerType7,239,0.987920,0.191262,45.711725
7,CustomerType8,564,1.059495,0.258637,145.871347
8,CustomerType9,927,1.327682,0.217489,201.612205
9,CustomerType10,366,1.628393,0.221076,80.913985
